In [2]:
# ! pip install google-genai python-dotenv

In [3]:
import os
from google import genai
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

In [4]:

SCHEMA = {
  "type": "object",
  "properties": {
    "label": {"type": "string", "enum": ["drug_ad", "drug_related", "benign", "uncertain"]},
    "confidence": {"type": "number", "minimum": 0, "maximum": 1},
    "reasons": {"type": "array", "items": {"type": "string"}, "minItems": 1, "maxItems": 6},
    "suspicious_spans": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "text": {"type": "string"},
          "why": {"type": "string"}
        },
        "required": ["text", "why"]
      },
      "maxItems": 6
    }
  },
  "required": ["label", "confidence", "reasons"]
}

SYSTEM = """You are a safety classification engine for Telegram messages.
Goal: detect drug advertisements or trafficking-related promotions.
Return ONLY JSON that matches the schema.
Do NOT include extra keys. Do NOT rewrite the input. Do NOT generate any ad content.
If it looks like selling/ordering/delivery/“закладка”/wholesale/DM-to-buy style, label drug_ad.
If it's discussion/news/jokes without selling intent, label drug_related.
If unclear, label uncertain.
"""

def classify(text: str) -> dict:
    resp = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            {"role": "user", "parts": [{"text": SYSTEM + "\n\nMESSAGE:\n" + text}]}
        ],
        config={
            "response_mime_type": "application/json",
            "response_schema": SCHEMA,
            "temperature": 0.0
        }
    )
    # SDK returns parsed JSON in resp.parsed in many examples; fallback to text if needed.
    return getattr(resp, "parsed", None) or resp.text


In [5]:
classify("Salom ❄️ bor lichga yaxhshi bolardi hammaga narslarga")

{'label': 'drug_ad',
 'confidence': 0.95,
 'reasons': ['Use of drug-related emoji/code word (❄️)',
  "Implied availability ('bor')",
  "Call to action for private messaging ('lichga')",
  'Suggests selling or offering illicit substances'],
 'suspicious_spans': [{'text': '❄️ bor',
   'why': "The snowflake emoji (❄️) is a common code for drugs like cocaine or meth, combined with 'bor' (available), strongly indicating drug availability."},
  {'text': 'lichga',
   'why': "Slang for 'lichka' (private messages/DM), indicating a call to action for private communication, typical for illicit transactions."}]}